In [2]:
import transformers
import pandas as pd
import numpy as np
import PIL
import os
import tqdm

In [3]:
items = pd.read_csv("../../data/processed_data/articles.csv", dtype={'article_id': str})
items.head()

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,section_no,section_name,garment_group_no,garment_group_name,detail_desc,season,number_of_purchases,sex,target_age_groupe,image_embedding
0,0108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.,0,10841,0,1,[[[1.9064132 1.9064132 1.8892884 ... 1.8207895...
1,0108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.,1,7250,0,1,[[[1.923538 1.923538 1.9064132 ... 1.7865399...
2,0108775051,108775,Strap top (1),253,Vest top,Garment Upper body,1010017,Stripe,11,Off White,...,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.,3,215,0,1,[[[1.9064132 1.8892884 1.8721637 ... 1.8036647...
3,0110065001,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,9,Black,...,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde...",3,1044,0,1,[[[1.8379141 1.8550389 1.8550389 ... 1.8036647...
4,0110065002,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,10,White,...,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde...",0,539,0,2,[[[1.8892884 1.8892884 1.8892884 ... 1.7865399...


In [6]:
item_dict = {}

In [17]:
data_dir = "../../data/raw_data/images"
dirs = os.listdir(data_dir)
cnt = 0
feature_extractor = transformers.DeiTFeatureExtractor.from_pretrained("facebook/deit-base-distilled-patch16-224")
for i in tqdm.tqdm(range(len(dirs))):
    class_path = os.path.join(data_dir, dirs[i])
    images = os.listdir(class_path)
    for image in images:
        try:
            input = PIL.Image.open(class_path + '/' + image)
            item_dict[image.replace(".jpg", "")] = feature_extractor(input, return_tensors="pt")['pixel_values'][0].numpy()
        except:
            cnt += 1

/opt/homebrew/lib/python3.10/site-packages/transformers/models/deit/feature_extraction_deit.py:28: FutureWarning: The class DeiTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use DeiTImageProcessor instead.
  warnings.warn(
100%|██████████| 86/86 [50:10<00:00, 35.00s/it]  


In [52]:
for i in items.article_id.unique():
    if not(i in item_dict.keys()):
        item_dict[i] = None

In [53]:
items["image_embedding"] = items.article_id.apply(lambda x: item_dict[x])
items.head()

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,section_no,section_name,garment_group_no,garment_group_name,detail_desc,season,number_of_purchases,sex,target_age_groupe,image_embedding
0,0108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.,0,10841,0,1,"[[[1.9064132, 1.9064132, 1.8892884, 1.9064132,..."
1,0108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.,1,7250,0,1,"[[[1.923538, 1.923538, 1.9064132, 1.923538, 1...."
2,0108775051,108775,Strap top (1),253,Vest top,Garment Upper body,1010017,Stripe,11,Off White,...,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.,3,215,0,1,"[[[1.9064132, 1.8892884, 1.8721637, 1.8892884,..."
3,0110065001,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,9,Black,...,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde...",3,1044,0,1,"[[[1.8379141, 1.8550389, 1.8550389, 1.8550389,..."
4,0110065002,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,10,White,...,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde...",0,539,0,2,"[[[1.8892884, 1.8892884, 1.8892884, 1.8892884,..."


In [63]:
mean = items["image_embedding"].mean()
mean

array([[[1.9164094, 1.9118924, 1.9082154, ..., 1.8623496, 1.8647721,
         1.8683699],
        [1.9152513, 1.9112766, 1.9074585, ..., 1.860359 , 1.8633745,
         1.8671111],
        [1.9135342, 1.9093267, 1.905713 , ..., 1.8593395, 1.861775 ,
         1.8656653],
        ...,
        [1.7740953, 1.7699671, 1.7646263, ..., 1.7399375, 1.744747 ,
         1.7494932],
        [1.7794898, 1.7748535, 1.7705003, ..., 1.7456772, 1.749625 ,
         1.7546283],
        [1.7859536, 1.7821788, 1.7777604, ..., 1.7508297, 1.7552737,
         1.7600667]],

       [[2.071772 , 2.0670514, 2.0629916, ..., 2.0167608, 2.0194662,
         2.02348  ],
        [2.0704262, 2.0662017, 2.06196  , ..., 2.0147257, 2.0179648,
         2.0220492],
        [2.0685492, 2.0639524, 2.0600016, ..., 2.0136662, 2.0162754,
         2.0205019],
        ...,
        [1.9187806, 1.9140763, 1.9081109, ..., 1.8836423, 1.8890461,
         1.8944874],
        [1.9253976, 1.9200335, 1.9149764, ..., 1.8901675, 1.894638 ,
   

In [92]:
items.loc[items.image_embedding.isna(), "image_embedding"] = items.loc[items.image_embedding.isna(), "image_embedding"].apply(lambda _: mean)
items.image_embedding.isna().sum()

0

In [94]:
items.to_csv('../../data/processed_data/articles.csv', index=False)